In [20]:
!pip install requests beautifulsoup4 sentence-transformers faiss-cpu nltk

In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to C:\Users\JISBIN
[nltk_data]     JEES\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [40]:
url = "https://www.indusind.bank.in/in/en/personal/rbi-notifications/settlement-of-claims-in-respect-of-deceased-customers.html"

html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

# Remove junk
for tag in soup(["script", "style", "nav", "footer", "header"]):
    tag.decompose()

text = soup.get_text(" ")
clean_text = " ".join(text.split())

print(clean_text[:800])


Settlement of Claims in respect of Deceased Customers Home RBI Notifications Settlement of Claims in respect of Deceased Customers Overview The Reserve Bank of India vide its circular No. DoR.MCS.REC.50/01.01.003/2025-26 dated 26 th Sep 2025 had issued detailed regulations on deceased claim settlement procedures and standardise the documentation to bring improvement in the quality of customer service. The nomination facility in deposit accounts, safe deposit lockers under the provisions of Sections 45ZA to ZF of the Banking Regulation Act, 1949 read with Section 56 of the Act ibid is intended to facilitate expeditious settlement of claims by banks upon death of a deceased customer and to minimise hardship caused to the family members. To streamline the procedures and standardise the docume


In [33]:
sentences = sent_tokenize(clean_text)

chunks = []
chunk = ""

for sent in sentences:
    if len(chunk) + len(sent) < 700:
        chunk += " " + sent
    else:
        chunks.append(chunk.strip())
        chunk = sent

chunks.append(chunk.strip())

print("Total chunks:", len(chunks))

Total chunks: 27


In [56]:
# for online pdf
import requests

#pdf_url = "https://icmt.unionbankofindia.co.in/deathclaim/Documents/Policy%20on%20Settlement%20of%20Death%20Claim%202025-26.pdf"
pdf_url="https://www.idbi.bank.in/pdf/Procedure-for-settlement.pdf"
pdf_path = "Procedure-for-settlement.pdf"

response = requests.get(pdf_url)
with open(pdf_path, "wb") as f:
    f.write(response.content)

print("PDF downloaded")


PDF downloaded


In [57]:
!pip install pdfplumber

In [58]:
#Extract text from PDF
import pdfplumber

all_text = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            all_text += text + "\n"

print(all_text[:1000])


Procedure for settlement of Deceased claims of Locker / Safe Custody Article
Nomination facility in Safe Deposit Lockers
Nomination facility is available in respect of lockers hired singly as well as jointly. In respect of
lockers in joint names, nomination can be made only when the mode of operation is stipulated
as ‘Jointly’. When a locker is hired by two or more persons with mode of operation stipulated as
‘Jointly’, each hirer may appoint a nominee on his/her behalf.
Where the lockers are hired jointly, on the death of any of the joint hirers, the contents of the
locker are allowed to be removed jointly by the nominee(s) and the survivor(s) after an inventory
is taken in the prescribed manner. In such a case, after such removal preceded by an inventory,
the nominee and surviving hirer(s) may still keep the entire contents with the bank, if they so
desire, by entering into a fresh contract of hiring a locker.
Sealed/closed packets found in locker while releasing them to the nominee 

In [59]:
#smart chugging
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(all_text)

chunks = []
chunk = ""

for sent in sentences:
    if len(chunk) + len(sent) < 700:
        chunk += " " + sent
    else:
        chunks.append(chunk.strip())
        chunk = sent

chunks.append(chunk.strip())

print("Total chunks:", len(chunks))


Total chunks: 26


[nltk_data] Downloading package punkt to C:\Users\JISBIN
[nltk_data]     JEES\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(chunks)


In [61]:
def retrieve_context(query, top_k=3):
    query_vec = model.encode([query])
    scores = cosine_similarity(query_vec, embeddings)[0]

    top_indices = scores.argsort()[-top_k:][::-1]

    return [(chunks[i], scores[i]) for i in top_indices]


In [62]:
def answer_question(query):
    results = retrieve_context(query)

    print("\n Retrieved Context:\n")
    for i, (text, score) in enumerate(results):
        print(f"Source {i+1} (score={round(score,2)}):\n{text[:400]}\n")

    answer = "As per Union Bank / RBI death claim policy:\n\n"

    for text, _ in results:
        answer += "- " + text[:300] + "\n"

    answer += "\n If information is missing, legal documents like succession certificate may be required."

    return answer


In [63]:
query = "If account holder died and nominee also died, who will receive the money?"

response = answer_question(query)

print("\n✅ FINAL ANSWER:\n")
print(response)



 Retrieved Context:

Source 1 (score=0.5600000023841858):
legal heirs and proof of death of the hirer.

Source 2 (score=0.5):
i) In case the nominee/legal heir(s)/survivor(s) is/are not in possession of the key, the
nominee/ legal heir(s)/survivor(s) shall give a letter to that effect and request the Bank to
arrange for breaking open the locker. A suitable date for breaking open the locker shall
be fixed and necessary charges for breaking open the locker shall be collected from the
nominee/legal heir(s)/survivor(s). The 

Source 3 (score=0.46000000834465027):
ISA as per the extant KYC Policy of the Bank,
Before permitting nominee to remove contents his/her/their authority and verification of proof
of the Safe Custody Articles, the Bank would of death of the depositor. Before permitting
prepare an inventory of the articles in the legal heir(s) to remove contents of a Safe
presence of nominee and two independent Custody Articles, the Bank would prepare a


✅ FINAL ANSWER:

As per Union 

In [64]:
query = "if account holder is died who is eligible for the locker operations"

response = answer_question(query)

print("\n✅ FINAL ANSWER:\n")
print(response)


 Retrieved Context:

Source 1 (score=0.699999988079071):
i) In case the nominee/legal heir(s)/survivor(s) is/are not in possession of the key, the
nominee/ legal heir(s)/survivor(s) shall give a letter to that effect and request the Bank to
arrange for breaking open the locker. A suitable date for breaking open the locker shall
be fixed and necessary charges for breaking open the locker shall be collected from the
nominee/legal heir(s)/survivor(s). The 

Source 2 (score=0.6899999976158142):
In case the locker was hired jointly with survivorship clause and the hirers
instructed that the access of the locker should be given over to ‘either or
survivor’, ‘anyone or survivor’ or ‘former or survivor’ or according to any other
survivorship clause, banks should follow the mandate in the event of the death of
one of more of the locker-hirers. The survivor/s may be allowed to continue the
sam

Source 3 (score=0.6700000166893005):
Procedure for settlement of Deceased claims of Locker / Safe Cus